# Segmenting and Clustering Neighborhoods in Toronto

Impoting libraries that we use in this work.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: ...working... done

# All requested packages already installed.



Sistem belirtilen yolu bulamıyor.


Starting scrapping, we are using BeautifulSoup library for scrapping this wikipedia page to optain the table. After investigation there is just one table tag in this html data. First we request the html of the page, than using the httml parser of the BS, parse the page. We have BS oject with wikipedia html now. 

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
wpp = requests.get(url)

In [4]:
soup = BeautifulSoup(wpp.content,'html.parser')

In [5]:
soup_table = soup.find('table')

Than we get every row in a loop and find all columns in every row, extract text inside it and append new list called table

In [6]:
table = []
for row in soup_table.find_all('tr'):
    subtable = []
    cell = row.find_all('td')
    for i in cell:
        subtable.append(i.find(text=True).rstrip("\n"))
    table.append(subtable)

Delete first empty row of the list than count raw data.

In [7]:
table.remove([])
len(table)

180

We are processing list according to rules shown in assignment page. 
* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [8]:
table_processed = []

In [9]:
for i in table:
    if (i[1]=='Not assigned' and i[2]=='Not assigned'):
        print('pass', end=" ")    
    elif i[2] == 'Not assigned':
        i[2] = i[1]
        table_processed.append(i)
        print("change and appended", end=" ")
    else:
        table_processed.append(i)
        print("appended", end=" ")

pass pass appended appended appended appended appended pass appended appended pass appended appended appended appended pass pass appended appended pass appended appended appended appended pass pass appended appended pass pass appended appended appended pass pass pass appended pass pass appended appended appended pass pass pass appended appended appended appended appended appended pass pass pass appended appended appended appended appended appended pass pass pass appended appended appended appended appended appended pass pass pass appended appended appended appended appended appended pass pass appended appended appended appended appended appended appended pass pass appended appended appended appended appended appended appended pass pass appended appended appended pass appended appended appended pass pass appended appended appended pass appended appended appended appended pass appended appended pass pass appended appended appended pass pass pass appended pass pass appended appended pass 

In [10]:
df = pd.DataFrame(table_processed,columns = ['PostalCode','Borough','Neighborhood'])

In [11]:
df.shape

(103, 3)

In [12]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Getting Locations from CSV

In [13]:
df_loc = pd.read_csv("Geospatial_Coordinates.csv")

In [14]:
df_loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
lat = []
lon = []

for pcode in df["PostalCode"]:
    lat.append(df_loc['Latitude'].loc[df_loc['Postal Code'] == pcode].values[0])
    lon.append(df_loc['Longitude'].loc[df_loc['Postal Code'] == pcode].values[0])

In [16]:
df['Latitude'] = lat
df['Longitude'] = lon

In [17]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [18]:
map = folium.Map(location=[43.651070, -79.347015], zoom_start=9)
for index, row in df.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=50,
        popup=row["Neighborhood"],
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    ).add_to(map)

map
    


In [19]:
map.save('index.html')